In [1]:
import os
import shutil
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
zip_path = '/content/drive/MyDrive/School UTSA/AI Grad/Fall 2024/Shared Projects File /Task01_BrainTumour.tar'

In [4]:
#nifti files are series of dicoms

In [5]:
import tarfile
destination_path = '/content/brain_tumor_data'

with tarfile.open(zip_path, 'r') as tar:
    tar.extractall(destination_path)

print("Decompression completed.")


Decompression completed.


In [6]:
# List the contents of the destination folder to get an overview
for root, dirs, files in os.walk(destination_path):
    level = root.replace(destination_path, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f in files:
        print(f"{sub_indent}{f}")

brain_tumor_data/
    Task01_BrainTumour/
        ._imagesTs
        ._dataset.json
        ._imagesTr
        ._labelsTr
        dataset.json
        imagesTs/
            BRATS_594.nii.gz
            BRATS_629.nii.gz
            BRATS_558.nii.gz
            BRATS_656.nii.gz
            BRATS_682.nii.gz
            BRATS_647.nii.gz
            BRATS_508.nii.gz
            BRATS_617.nii.gz
            BRATS_608.nii.gz
            BRATS_576.nii.gz
            BRATS_612.nii.gz
            BRATS_747.nii.gz
            BRATS_673.nii.gz
            BRATS_552.nii.gz
            BRATS_621.nii.gz
            BRATS_731.nii.gz
            BRATS_733.nii.gz
            BRATS_602.nii.gz
            BRATS_551.nii.gz
            BRATS_648.nii.gz
            BRATS_573.nii.gz
            BRATS_540.nii.gz
            ._BRATS_739.nii.gz
            ._BRATS_490.nii.gz
            ._BRATS_501.nii.gz
            BRATS_711.nii.gz
            BRATS_587.nii.gz
            BRATS_538.nii.gz
            BRATS_729

In [7]:
import os
import shutil

# Define the original and new directories
original_imagesTr_path = '/content/brain_tumor_data/Task01_BrainTumour/imagesTr'
original_labelsTr_path = '/content/brain_tumor_data/Task01_BrainTumour/labelsTr'
subsample_dir = '/content/brain_tumor_data/Subsample Images'
subsample_imagesTr_path = os.path.join(subsample_dir, 'imagesTr')
subsample_labelsTr_path = os.path.join(subsample_dir, 'labelsTr')

# Create the new folders
os.makedirs(subsample_imagesTr_path, exist_ok=True)
os.makedirs(subsample_labelsTr_path, exist_ok=True)

# Copy the first two valid files (excluding hidden/system files) from imagesTr
image_files = sorted([f for f in os.listdir(original_imagesTr_path) if not f.startswith('._')])[:2]  # Get the first two valid files
for file_name in image_files:
    src_path = os.path.join(original_imagesTr_path, file_name)
    dest_path = os.path.join(subsample_imagesTr_path, file_name)
    shutil.copy(src_path, dest_path)

# Copy the first two valid files (excluding hidden/system files) from labelsTr
label_files = sorted([f for f in os.listdir(original_labelsTr_path) if not f.startswith('._')])[:2]  # Get the first two valid files
for file_name in label_files:
    src_path = os.path.join(original_labelsTr_path, file_name)
    dest_path = os.path.join(subsample_labelsTr_path, file_name)
    shutil.copy(src_path, dest_path)

print("Subsample Images with the first two files copied for each training set created successfully.")

# Print the contents of the subsample directory to verify
print("\nContents of the Subsample Images Directory:")
for root, dirs, files in os.walk(subsample_dir):
    level = root.replace(subsample_dir, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f in files:
        print(f"{sub_indent}{f}")


Subsample Images with the first two files copied for each training set created successfully.

Contents of the Subsample Images Directory:
Subsample Images/
    labelsTr/
        BRATS_002.nii.gz
        BRATS_001.nii.gz
    imagesTr/
        BRATS_002.nii.gz
        BRATS_001.nii.gz


In [8]:
'''
Monai preprocessing  steps:
  create random crops windows
  take crop that they used and pass it for the training and then slide the window across
the entire image


For this code well be creating by oursels this window but let the width and height be the enitre image, resize the images
512x512, we will resize the images. we will remove the black background as much as we cna


'''

'\nMonai preprocessing  steps:\n  create random crops windows\n  take crop that they used and pass it for the training and then slide the window across\nthe entire image\n\n\nFor this code well be creating by oursels this window but let the width and height be the enitre image, resize the images\n512x512, we will resize the images. we will remove the black background as much as we cna\n  \n\n'

In [9]:
'''
Convert from Nifti to Dicom
'''
import nibabel as nib

# Load a 4D NIfTI image
img_path = '/content/brain_tumor_data/Subsample Images/imagesTr/BRATS_001.nii.gz'  # Replace with your file path
img = nib.load(img_path)
data = img.get_fdata()
print("Data shape:", data.shape)  # Example output: (240, 240, 155, 4)



Data shape: (240, 240, 155, 4)


In [10]:
T1 = data[:, :, :, 0]
T1Gd = data[:, :, :, 1]
T2 = data[:, :, :, 2]
FLAIR = data[:, :, :, 3]

print("T1 shape:", T1.shape)
print("T1Gd shape:", T1Gd.shape)
print("T2 shape:", T2.shape)
print("FLAIR shape:", FLAIR.shape)


T1 shape: (240, 240, 155)
T1Gd shape: (240, 240, 155)
T2 shape: (240, 240, 155)
FLAIR shape: (240, 240, 155)


In [11]:
!pip install pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.4 MB/s eta 0:00:00


In [12]:
import os
import nibabel as nib
import numpy as np
import pydicom
from pydicom.dataset import Dataset, FileDataset
from pydicom.uid import generate_uid

def save_volume_to_dicom(volume, output_dir, modality_name="Modality"):
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

    # Loop through each slice along the z-axis
    for i in range(volume.shape[2]):
        slice_data = volume[:, :, i]  # Extract the 2D slice

        # Create a new DICOM dataset for the slice
        file_meta = pydicom.Dataset()
        ds = FileDataset('', {}, file_meta=file_meta, preamble=b"\0" * 128)

        # Populate necessary DICOM metadata
        ds.PatientName = "SamplePatient"
        ds.PatientID = "123456"
        ds.Modality = "MR"
        ds.SeriesDescription = modality_name
        ds.StudyInstanceUID = generate_uid()
        ds.SeriesInstanceUID = generate_uid()
        ds.SOPInstanceUID = generate_uid()
        ds.InstanceNumber = i + 1

        # Set pixel data and shape information
        ds.Rows, ds.Columns = slice_data.shape
        ds.PhotometricInterpretation = "MONOCHROME2"
        ds.SamplesPerPixel = 1
        ds.BitsAllocated = 16
        ds.BitsStored = 16
        ds.HighBit = 15
        ds.PixelRepresentation = 1
        ds.PixelData = slice_data.astype(np.int16).tobytes()

        # Save the DICOM file
        dicom_filename = os.path.join(output_dir, f"{modality_name}_slice_{i+1:03d}.dcm")
        ds.save_as(dicom_filename)
        print(f"Saved DICOM: {dicom_filename}")

def process_directory_of_nifti_files(nifti_dir, main_dicom_dir, isTraining=True):
    # Determine the subfolder name based on isTraining
    subfolder_name = "imagesTr" if isTraining else "labelsTr"

    # Loop through all NIfTI files in the specified directory
    for nifti_filename in os.listdir(nifti_dir):
        if nifti_filename.endswith('.nii.gz'):  # Only process .nii.gz files
            # Full path to the NIfTI file
            nifti_path = os.path.join(nifti_dir, nifti_filename)

            # Extract the base file name (e.g., "BRATS_001" from "BRATS_001.nii.gz")
            file_name = os.path.splitext(os.path.splitext(nifti_filename)[0])[0]  # Remove both .nii and .gz

            # Set up output directory for DICOM files under the appropriate subfolder (imagesTr or labelsTr)
            output_dir = os.path.join(main_dicom_dir, "DICOM_IMAGES", subfolder_name, file_name)
            os.makedirs(output_dir, exist_ok=True)

            # Load the NIfTI file
            img = nib.load(nifti_path)
            data = img.get_fdata()

            # Extract the T1 modality (assuming it's in the first channel for training images)
            # For label data, we may only have a single channel, but we can still use the same code
            modality_volume = data[:, :, :, 0] if isTraining else data  # For labels, we assume data is 3D

            # Save the volume as DICOM slices
            modality_name = "T1" if isTraining else "Label"
            save_volume_to_dicom(modality_volume, output_dir, modality_name=modality_name)
            print(f"Processed and saved DICOMs for {file_name} in {subfolder_name}")

# Example usage for training images:
nifti_dir_train = '/content/brain_tumor_data/Subsample Images/imagesTr'  # Directory containing training NIfTI files
main_dicom_dir = '/content/dicom_output'  # Main output directory for DICOM files
process_directory_of_nifti_files(nifti_dir_train, main_dicom_dir, isTraining=True)

# Example usage for label images:
nifti_dir_labels = '/content/brain_tumor_data/Subsample Images/labelsTr'  # Directory containing label NIfTI files
process_directory_of_nifti_files(nifti_dir_labels, main_dicom_dir, isTraining=False)


Saved DICOM: /content/dicom_output/DICOM_IMAGES/imagesTr/BRATS_002/T1_slice_001.dcm
Saved DICOM: /content/dicom_output/DICOM_IMAGES/imagesTr/BRATS_002/T1_slice_002.dcm
Saved DICOM: /content/dicom_output/DICOM_IMAGES/imagesTr/BRATS_002/T1_slice_003.dcm
Saved DICOM: /content/dicom_output/DICOM_IMAGES/imagesTr/BRATS_002/T1_slice_004.dcm
Saved DICOM: /content/dicom_output/DICOM_IMAGES/imagesTr/BRATS_002/T1_slice_005.dcm
Saved DICOM: /content/dicom_output/DICOM_IMAGES/imagesTr/BRATS_002/T1_slice_006.dcm
Saved DICOM: /content/dicom_output/DICOM_IMAGES/imagesTr/BRATS_002/T1_slice_007.dcm
Saved DICOM: /content/dicom_output/DICOM_IMAGES/imagesTr/BRATS_002/T1_slice_008.dcm
Saved DICOM: /content/dicom_output/DICOM_IMAGES/imagesTr/BRATS_002/T1_slice_009.dcm
Saved DICOM: /content/dicom_output/DICOM_IMAGES/imagesTr/BRATS_002/T1_slice_010.dcm
Saved DICOM: /content/dicom_output/DICOM_IMAGES/imagesTr/BRATS_002/T1_slice_011.dcm
Saved DICOM: /content/dicom_output/DICOM_IMAGES/imagesTr/BRATS_002/T1_slice_

In [13]:
import os

def check_directory_counts(images_dir, labels_dir):
    # Get a list of all subdirectories in each directory
    images_subdirs = [d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))]
    labels_subdirs = [d for d in os.listdir(labels_dir) if os.path.isdir(os.path.join(labels_dir, d))]

    # Check if the number of directories matches
    if len(images_subdirs) == len(labels_subdirs):
        print(f"Match: Both directories have {len(images_subdirs)} subdirectories.")
    else:
        print(f"Mismatch: imagesTr has {len(images_subdirs)} subdirectories, while labelsTr has {len(labels_subdirs)} subdirectories.")

    # Optionally, print any directories missing in one or the other
    missing_in_labels = set(images_subdirs) - set(labels_subdirs)
    missing_in_images = set(labels_subdirs) - set(images_subdirs)

    if missing_in_labels:
        print("Directories in imagesTr missing from labelsTr:", missing_in_labels)
    if missing_in_images:
        print("Directories in labelsTr missing from imagesTr:", missing_in_images)

# Example usage
images_dir = '/content/dicom_output/DICOM_IMAGES/imagesTr'
labels_dir = '/content/dicom_output/DICOM_IMAGES/labelsTr'
check_directory_counts(images_dir, labels_dir)


Match: Both directories have 2 subdirectories.


In [14]:
#Create groups with number of slices and convert groups into nifti files
#64 slices
#for each group we will create a folder
#convert 64 slice into nifti files

#create folder for labels and one for the images
#create folder called dicom_groups
#create subfolders called images and labels

in_path = '/content/dicom_output/DICOM_IMAGES/labelsTr'
out_path = '/content/dicom_output/DICOM_IMAGES/dicom_groups/labels'


In [15]:
#need to create and restore paths
!pip install glob2

In [17]:
!pip install pytest-shutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 3.2 MB/s eta 0:00:00


In [19]:
from glob import glob
import shutil
import os

In [22]:
everything_in_path = glob(in_path + '/*')

print(everything_in_path[0])

/content/dicom_output/DICOM_IMAGES/labelsTr/BRATS_002


In [23]:
num_slices = 64
everything_in_path = glob(in_path + '/*')
for patient in everything_in_path:
  patient_name = os.path.basename(os.path.normpath(patient))
  number_folders = int(len(glob(patient+'/*'))/num_slices)

  for i in range(number_folders):
    #create folder


BRATS_001
BRATS_002
